Setting environment and elements

In [2]:
import sys
import pdb
import time

import numpy as np
import pandas as pd
from datetime import timedelta




In [3]:
BANDS = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7','ST_B10']

Reading data

In [4]:
df2 = pd.read_csv('landsat8RSdata.csv')

Calculating SurfSurface reflectance and surface temperature 

In [5]:
#Defining function to calculate surface reflectance
def refl(x):
    return x*0.0000275-0.2
#Defining function to calculate surface temperature
def temp(x):
    return x*0.00341802+149-273.15

#Calculating surface reflectance and temperature for each band in the dataset
for band in ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']:
    df2[band] = df2[band].apply(lambda x: refl(x))
    df2[df2[band]<0] = np.nan
    

df2['ST_B10'] = df2['ST_B10'].apply(lambda x: temp(x))
df2[df2['ST_B10']<0] = np.nan
df2 = df2.dropna()

df2.to_csv('surfacedata.csv')

In [6]:
df2

,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,ST_B10,rect,date
0,0.014430,0.022561,0.042909,0.028666,0.132401,0.063604,0.036339,28.236100,0.0,2017-01-08
1,0.010011,0.021636,0.053264,0.041710,0.247956,0.118310,0.059735,20.349233,0.0,2017-02-25
4,0.004413,0.016695,0.045400,0.034215,0.213877,0.110092,0.054481,31.826869,0.0,2017-04-14
5,0.015956,0.034065,0.073066,0.055916,0.289974,0.143563,0.070895,17.204029,0.0,2017-04-30
6,0.006667,0.025957,0.064698,0.046880,0.242143,0.127363,0.072002,14.565461,0.0,2017-05-16
...,...,...,...,...,...,...,...,...,...,...
849748,0.027480,0.042811,0.071728,0.075709,0.212997,0.181295,0.109750,37.107234,7687.0,2021-08-31
849749,0.029714,0.038633,0.065238,0.075375,0.219932,0.184851,0.108710,41.142434,7687.0,2021-09-16
849751,0.031124,0.042090,0.071371,0.082625,0.227165,0.192097,0.114457,44.930045,7687.0,2021-10-18
849752,0.031016,0.041101,0.070433,0.081527,0.235380,0.191339,0.102384,41.919312,7687.0,2021-11-03


Calculating index

Defining index function

In [7]:
def EVI(NIR, Red, Blue):
    return 2.5 * (NIR - Red) / (NIR + 6 * Red - 7.5 * Blue + 1)
def NDVI(NIR, Red):
    return (NIR - Red) / (NIR + Red)
def GNDVI(NIR, Green):
    return (NIR - Green) / (NIR + Green)
def NDMI(NIR, SWIR1):
    return (NIR - SWIR1) / (NIR + SWIR1)
def SAVI(NIR, Red, L=0.33):
    return ((NIR - Red) / (NIR + Red + L)) * (1 + L)
def MSAVI(NIR, Red):
    return (1/2) * (2 * (NIR +1) - np.sqrt(np.square(2 * NIR + 1) - 8 * (NIR - Red)))
def CIGreen(NIR, Green):
    return (NIR / Green)-1
def NBR(NIR, SWIR1):
    return (NIR - SWIR1) / (NIR+ SWIR1)

Rename

In [8]:
df3 = df2.copy()
df3 = df3.rename(columns={'SR_B1':'Aerosol',
                    'SR_B2':'Blue',
                    'SR_B3':'Green',
                    'SR_B4':'Red',
                    'SR_B5':'NIR',
                    'SR_B6':'SWIR1',
                    'SR_B7':'SWIR2',
                    'ST_B10':'Temperature',
                    'rect':'rect_id'
                    })
df3 = df3[['rect_id', 'date','Aerosol','Blue','Green','Red','NIR','SWIR1','SWIR2','Temperature']]
df3

,rect_id,date,Aerosol,Blue,Green,Red,NIR,SWIR1,SWIR2,Temperature
0,0.0,2017-01-08,0.014430,0.022561,0.042909,0.028666,0.132401,0.063604,0.036339,28.236100
1,0.0,2017-02-25,0.010011,0.021636,0.053264,0.041710,0.247956,0.118310,0.059735,20.349233
4,0.0,2017-04-14,0.004413,0.016695,0.045400,0.034215,0.213877,0.110092,0.054481,31.826869
5,0.0,2017-04-30,0.015956,0.034065,0.073066,0.055916,0.289974,0.143563,0.070895,17.204029
6,0.0,2017-05-16,0.006667,0.025957,0.064698,0.046880,0.242143,0.127363,0.072002,14.565461
...,...,...,...,...,...,...,...,...,...,...
849748,7687.0,2021-08-31,0.027480,0.042811,0.071728,0.075709,0.212997,0.181295,0.109750,37.107234
849749,7687.0,2021-09-16,0.029714,0.038633,0.065238,0.075375,0.219932,0.184851,0.108710,41.142434
849751,7687.0,2021-10-18,0.031124,0.042090,0.071371,0.082625,0.227165,0.192097,0.114457,44.930045
849752,7687.0,2021-11-03,0.031016,0.041101,0.070433,0.081527,0.235380,0.191339,0.102384,41.919312


Calculating

In [9]:
df3['EVI'] = df3.apply(lambda x: EVI(x.NIR, x.Red, x.Blue), axis=1)
df3['NDVI'] = df3.apply(lambda x: NDVI(x.NIR, x.Red), axis=1)
df3['GNDVI'] = df3.apply(lambda x: GNDVI(x.NIR, x.Green), axis=1)
df3['NDMI'] = df3.apply(lambda x: NDMI(x.NIR, x.SWIR1), axis=1)
df3['SAVI'] = df3.apply(lambda x: SAVI(x.NIR, x.Red, L=0.33), axis=1)
df3['MSAVI'] = df3.apply(lambda x: MSAVI(x.NIR, x.Red), axis=1)
df3['CIGreen'] = df3.apply(lambda x: CIGreen(x.NIR, x.Green), axis=1)
df3['NBR'] = df3.apply(lambda x: NBR(x.NIR, x.SWIR1), axis=1)


Data cleaning

In [10]:
df3[df3['EVI'] > 1] = np.nan
df3[df3['EVI'] < -1] = np.nan
df3[df3['MSAVI'] > 1] = np.nan
df3[df3['MSAVI'] < -1] = np.nan
df3 = df3.dropna()

Output index data

In [11]:
df3['rect_id'] = df3['rect_id'].apply(lambda x: int(x))
df3.to_csv('index.csv', index=False)

C:\Users\32429\AppData\Local\Temp\ipykernel_17416\508142145.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['rect_id'] = df3['rect_id'].apply(lambda x: int(x))


In [12]:
df3.describe

<bound method NDFrame.describe of         rect_id        date   Aerosol      Blue     Green       Red       NIR   
0             0  2017-01-08  0.014430  0.022561  0.042909  0.028666  0.132401  \
1             0  2017-02-25  0.010011  0.021636  0.053264  0.041710  0.247956   
4             0  2017-04-14  0.004413  0.016695  0.045400  0.034215  0.213877   
5             0  2017-04-30  0.015956  0.034065  0.073066  0.055916  0.289974   
6             0  2017-05-16  0.006667  0.025957  0.064698  0.046880  0.242143   
...         ...         ...       ...       ...       ...       ...       ...   
849748     7687  2021-08-31  0.027480  0.042811  0.071728  0.075709  0.212997   
849749     7687  2021-09-16  0.029714  0.038633  0.065238  0.075375  0.219932   
849751     7687  2021-10-18  0.031124  0.042090  0.071371  0.082625  0.227165   
849752     7687  2021-11-03  0.031016  0.041101  0.070433  0.081527  0.235380   
849755     7687  2021-12-21  0.017995  0.035184  0.071927  0.070677  0.2219